In [9]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

spark.conf.set("sprk.sql.parquet.vorder.enabled", "true")
spark.conf.set("sprk.microsoft.delta.optimizeWrite.enable", "true")
spark.conf.set("sprk.microsoft.delta.optimizeWrite.binSize", "1073741824")



StatementMeta(, d4c3c0ae-143e-43d9-b031-04830df707b0, 11, Finished, Available)

## **Load Remote Demo NYC Yellow Taxi data**

In [15]:
# Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = "r"

# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set(
  'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
  blob_sas_token)
print('Remote blob path: ' + wasbs_path)

# SPARK read parquet, note that it won't load any data yet by now
df = spark.read.parquet(wasbs_path)
print('Register the DataFrame as a SQL temporary view: source')
df.createOrReplaceTempView('source')

# Display top 10 rows
print('Displaying top 10 rows: ')
display(spark.sql('SELECT * FROM source LIMIT 10'))



StatementMeta(, d4c3c0ae-143e-43d9-b031-04830df707b0, 17, Finished, Available)

Remote blob path: wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow
Register the DataFrame as a SQL temporary view: source
Displaying top 10 rows: 


SynapseWidget(Synapse.DataFrame, ffdc2145-2237-4a2b-bf26-14e473b78aae)

In [ ]:
# display(spark.sql('SELECT min(tpepPickupDateTime), max(tpepPickupDateTime) FROM source LIMIT 10'))

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, f89e0deb-8c71-4931-9b0d-d244ba46a943)

## **Save Dataframe to Lakehouse Files Area**

In [16]:

df.write.mode("overwrite").format("parquet").partitionBy("puYear").save("Files/" + "rawnyctlcYellow")

StatementMeta(, d4c3c0ae-143e-43d9-b031-04830df707b0, 18, Finished, Available)

## **Load from Lakehouse Files Area to Bronze Table then run Optimize to compact files**

In [20]:
df = spark.read.parquet("Files/rawnyctlcYellow")
display(df.limit(20))

StatementMeta(, d4c3c0ae-143e-43d9-b031-04830df707b0, 22, Finished, Available)

SynapseWidget(Synapse.DataFrame, ba576179-58ae-4e42-a595-82508cb4e107)

needed to increase the Fabric capacity to F64 and confirm Spark pools allocation. 

In [21]:
df.write.mode("overwrite").format("delta").save("Tables/" + "bronze_nyctlcYellow")

StatementMeta(, d4c3c0ae-143e-43d9-b031-04830df707b0, 23, Finished, Available)

In [22]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "Tables/" + "bronze_nyctlcYellow")

deltaTable.optimize().executeCompaction()

StatementMeta(, d4c3c0ae-143e-43d9-b031-04830df707b0, 24, Finished, Available)

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de